In [1]:
import os
#os.environ["PATH"] = "=/usr/local/cuda-12.2/bin:$PATH"
#os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.2/lib64:$LD_LIBRARY_PATH"
#os.environ["CUDA_HOME"] = "/usr/local/cuda-12.2"
os.environ["TF_USE_LEGACY_KERAS"] = "0"

import tensorflow as tf

from tensorflow.keras import layers, models, activations
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import hls4ml
from tqdm import tqdm
tf.__version__

2024-06-13 09:44:43.972498: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 09:44:43.997412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 09:44:43.997438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 09:44:43.998120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 09:44:44.002500: I tensorflow/core/platform/cpu_feature_guar

/home/rikpi/.local/lib/python3.11/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


'2.15.1'

In [2]:
#Set sample number
sample_num = 300

dataset_dict={300:'./dataset.npy', 500:'./dataset_500.npy', 800:'./dataset_800.npy', 1000:'./dataset_1000.npy'}


# Load the data
with open (dataset_dict[sample_num],'rb') as f:
  X_train = np.load(f,allow_pickle=True)
  Y_train = np.load(f,allow_pickle=True)
  X_val = np.load(f,allow_pickle=True)
  Y_val = np.load(f,allow_pickle=True)
  X_test =np.load(f,allow_pickle=True)
  Y_test =np.load(f,allow_pickle=True)

num_classes = len(np.unique(Y_train))
y_train_onehot = tf.keras.utils.to_categorical(Y_train, num_classes=num_classes)
y_val_onehot = tf.keras.utils.to_categorical(Y_val, num_classes=num_classes)
y_test_onehot = tf.keras.utils.to_categorical(Y_test, num_classes=num_classes)

In [7]:
# MOHAMAD 1DCNN CLASS
class CNN_1D():
    """
    CNN_1D: Class to train a 1D Convolutional Neural Network
    Attributes:
    filters: List of filters for the convolutional layers
    kernels: List of kernel sizes for the convolutional layers
    window: Window size for the input data
    n_folds: Number of folds for cross-validation
    channels: Number of channels in the input data
    batch_size: Batch size for training
    epochs: Number of epochs for training
    learning_rate: Learning rate for the optimizer
    optimizer: Optimizer for training
    loss: Loss function for training
    metrics: Metrics for training
    callbacks: Callbacks for training
    norm_dict: Dictionary with normalization parameters
    train: Method to train the model
    """
    def __init__(self,
                 X_train,
                 y_train,
                 X_val,
                 y_val,
                 X_test,
                 y_test):
        self.filters = [[8], [16],[32],[64]]
        self.kernels = [2, 3, 4, 5]
        self.batch_size = 64
        self.epochs = 300
        self.learning_rate = 0.001
        self.n_classes = 8
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.loss = 'categorical_crossentropy'
        self.metrics = ['accuracy']
        self.callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
                         # tf.keras.callbacks.LearningRateScheduler(schedule=lambda epoch, lr: lr * 0.5 if epoch % 5 == 0 and epoch != 0 and lr>1e-4 else lr)]

        # Assign the training, validation, and testing data
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test
        ########## Train ################
        self.train()

    def __create_model(self, filters, kernel, n_classes, verbose=1):
        model = tf.keras.models.Sequential()
        for i, f in enumerate(filters):
            if i == 0:
                model.add(tf.keras.layers.Conv1D(f, kernel, input_shape=(self.X_train.shape[1:]), padding='same'))
                #model.add(tf.keras.layers.BatchNormalization())
                model.add(tf.keras.layers.Activation('relu'))
            else:
                model.add(tf.keras.layers.Conv1D(f, kernel, activation='relu', padding='same'))
                #model.add(tf.keras.layers.BatchNormalization())
                model.add(tf.keras.layers.Activation('relu'))

            model.add(tf.keras.layers.MaxPooling1D(2))
        # model.add(tf.keras.layers.GlobalAveragePooling1D())
        # model.add(tf.keras.layers.AveragePooling1D(pool_size=150))
        model.add(layers.Flatten())
        #model.add(tf.keras.layers.Dense(32, activation='relu'))
        model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))
        if verbose:
            model.summary()
        return model
    def train(self):
        best_val_acc = 0
        best_f=None
        best_k= None
        initial_optimizer_config = self.optimizer.get_config()
        with tqdm(total=len(self.filters) * len(self.kernels), desc="Training Progress", unit="iteration") as pbar:
            for f in self.filters:
                for k in self.kernels:
                    model = self.__create_model(f, k, self.n_classes, verbose=0)
                    optimizer = type(self.optimizer).from_config(initial_optimizer_config)
                    model.compile(optimizer=optimizer, loss=self.loss, metrics=self.metrics)
                    history = model.fit(self.X_train,self.y_train, epochs=self.epochs, batch_size=self.batch_size, validation_data=(self.X_val, self.y_val),callbacks=self.callbacks, verbose=0)
                    if history.history['val_accuracy'][-1] > best_val_acc:
                        # print('best val acc:', history.history['val_accuracy'][-1])
                        best_val_acc = history.history['val_accuracy'][-1]
                        model.save(f"model_{f}_{k}.h5")
                        best_f = f
                        best_k = k
                    pbar.update(1)
        best_model = tf.keras.models.load_model(f"model_{best_f}_{best_k}.h5")
        # best_model.compile(optimizer=self.optimizer, loss=self.loss, metrics=self.metrics)
        _, train_accuracy = best_model.evaluate(self.X_train,self.y_train, verbose=0)
        _, test_accuracy = best_model.evaluate(self.X_test,self.y_test, verbose=0)
        _, val_accuracy = best_model.evaluate(self.X_val,self.y_val, verbose=0)
        print(f"best_f: {best_f},best_k: {best_k}")
        print("Best model")
        best_model.summary()
        print("Train_accuracy:",train_accuracy,"Val accuracy:", val_accuracy, " Test Accuracy:", test_accuracy)

In [8]:
# TRAIN 1DCNN CLASS
model = CNN_1D(X_train, y_train_onehot, X_val, y_val_onehot, X_test, y_test_onehot)

Training Progress: 100%|██████████| 16/16 [10:28<00:00, 39.29s/iteration]


best_f: [64],best_k: 5
Best model
Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_51 (Conv1D)          (None, 300, 64)           2624      
                                                                 
 activation_51 (Activation)  (None, 300, 64)           0         
                                                                 
 max_pooling1d_51 (MaxPooli  (None, 150, 64)           0         
 ng1D)                                                           
                                                                 
 flatten_47 (Flatten)        (None, 9600)              0         
                                                                 
 dense_47 (Dense)            (None, 8)                 76808     
                                                                 
Total params: 79432 (310.28 KB)
Trainable params: 79432 (310.28 KB)
Non-trainable par

In [25]:
input_shape = (sample_num,8)
num_classes = 8
model = models.Sequential()
model.add(layers.InputLayer(input_shape=input_shape))
model.add(tf.keras.layers.Conv1D(32,8, activation='relu', padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.LSTM(12))

model.add(layers.Flatten())
model.add(layers.Dense(num_classes, activation='softmax', name='output_dense'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_59 (Conv1D)          (None, 300, 32)           2080      
                                                                 
 batch_normalization_37 (Ba  (None, 300, 32)           128       
 tchNormalization)                                               
                                                                 
 lstm_7 (LSTM)               (None, 12)                2160      
                                                                 
 flatten_54 (Flatten)        (None, 12)                0         
                                                                 
 output_dense (Dense)        (None, 8)                 104       
                                                                 
Total params: 4472 (17.47 KB)
Trainable params: 4408 (17.22 KB)
Non-trainable params: 64 (256.00 Byte)
________________

In [26]:

batch_size = 128

early_stopping = EarlyStopping(monitor='val_loss', patience=20)

history =model.fit(X_train.astype(np.float32),y_train_onehot, epochs=200, batch_size=batch_size,
validation_data=(X_val.astype(np.float32),y_val_onehot), callbacks=[early_stopping ])  

Epoch 1/200
215/215 [==============================] - 11s 27ms/step - loss: 1.8734 - accuracy: 0.2582 - val_loss: 1.7180 - val_accuracy: 0.3093
Epoch 2/200
215/215 [==============================] - 5s 23ms/step - loss: 1.5907 - accuracy: 0.3493 - val_loss: 1.5929 - val_accuracy: 0.3433
Epoch 3/200
215/215 [==============================] - 5s 22ms/step - loss: 1.5773 - accuracy: 0.3546 - val_loss: 1.7435 - val_accuracy: 0.3174
Epoch 4/200
215/215 [==============================] - 5s 22ms/step - loss: 1.4664 - accuracy: 0.4011 - val_loss: 1.3082 - val_accuracy: 0.4529
Epoch 5/200
215/215 [==============================] - 5s 21ms/step - loss: 1.4209 - accuracy: 0.4350 - val_loss: 1.3139 - val_accuracy: 0.4529
Epoch 6/200
215/215 [==============================] - 4s 21ms/step - loss: 1.2097 - accuracy: 0.5126 - val_loss: 1.1778 - val_accuracy: 0.5273
Epoch 7/200
215/215 [==============================] - 4s 21ms/step - loss: 1.1026 - accuracy: 0.5579 - val_loss: 1.0103 - val_accuracy

In [27]:
model.save('try_CRNN_LSTM.keras')